In [3]:
from skimage.metrics import mean_squared_error, structural_similarity
from skimage import io, color

def image_similarity(image1_path, image2_path):
    image1 = io.imread(image1_path)
    image2 = io.imread(image2_path)

    # Ensure both images have the same dimensions and color channels
    image1 = color.rgb2gray(image1)
    image2 = color.rgb2gray(image2)

    # Calculate MSE and RMSE
    mse = mean_squared_error(image1, image2)
    rmse = mse ** 0.5

    # Calculate SSIM
    ssim = structural_similarity(image1, image2)

    return mse, rmse, ssim

# Example usage
image1_path = "photos/from_facebook.jpg"
image2_path = "photos/from_gmail.jpg"
mse, rmse, ssim = image_similarity(image1_path, image2_path)
print("MSE:", mse)
print("RMSE:", rmse)
print("SSIM:", ssim)

MSE: 0.00023658293232054598
RMSE: 0.015381252625210536
SSIM: 0.9857939398187032


In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model

# Define the Siamese Network architecture
def create_siamese_network(input_shape):
    input_image = layers.Input(shape=input_shape)
    base_model = keras.applications.ResNet50(weights='imagenet', include_top=False, input_tensor=input_image)

    # Flatten the output from the base model
    flatten_layer = layers.Flatten()(base_model.output)

    # Fully connected layers for feature embedding
    fc1 = layers.Dense(512, activation='relu')(flatten_layer)
    fc2 = layers.Dense(512, activation='relu')(fc1)

    # Final embedding output
    embedding = layers.Dense(256)(fc2)

    # Create the siamese model
    siamese_model = Model(inputs=input_image, outputs=embedding)
    return siamese_model

# Function to calculate similarity between two embeddings
def similarity_metric(embedding1, embedding2):
    # Flatten the arrays to 1D
    embedding1 = np.squeeze(embedding1)
    embedding2 = np.squeeze(embedding2)

    # Calculate dot product
    dot_product = np.dot(embedding1, embedding2)

    # Calculate the norms of the arrays
    norm_embedding1 = np.linalg.norm(embedding1)
    norm_embedding2 = np.linalg.norm(embedding2)

    # Calculate the similarity score
    similarity_score = dot_product / (norm_embedding1 * norm_embedding2)

    return similarity_score

# Load images from paths
def load_image(image_path):
    image = keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image = keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    return image

# Load images from paths
image1_path = "photos/onedrive_unedited.jpg"
image2_path = "photos/edited.png"

image1 = load_image(image1_path)
image2 = load_image(image2_path)

# Create the Siamese Network
input_shape = (224, 224, 3)
siamese_model = create_siamese_network(input_shape)

# # Load pre-trained weights (optional)
# siamese_model.load_weights("path/to/pretrained_weights.h5")

# Get the embeddings for the two images
embedding1 = siamese_model.predict(image1)
embedding2 = siamese_model.predict(image2)

# Calculate similarity metric
similarity_score = similarity_metric(embedding1, embedding2)

print("Similarity Score:", similarity_score)



1/1 [==============================] - 0s 124ms/step
Similarity Score: 0.7099204
